### Option 1

In [29]:
# Importing the necessary libraries
import re
import json
import requests

from bs4 import BeautifulSoup

import pandas as pd

In [30]:
# We'll start by scraping the data of a single match
url = 'https://understat.com/match/27021'

response = requests.get(url)
print(response.status_code)

200


In [31]:
soup = BeautifulSoup(response.content, 'html.parser')
scripts = soup.find_all('script')
scripts

[<script>
 			var THEME = localStorage.getItem("theme") || 'DARK';
 			document.body.className = "theme-" + THEME.toLowerCase();
 		</script>,
 <script>
 	var shotsData 	= JSON.parse('\x7B\x22h\x22\x3A\x5B\x7B\x22id\x22\x3A\x22590240\x22,\x22minute\x22\x3A\x2212\x22,\x22result\x22\x3A\x22BlockedShot\x22,\x22X\x22\x3A\x220.79\x22,\x22Y\x22\x3A\x220.7280000305175781\x22,\x22xG\x22\x3A\x220.02347392961382866\x22,\x22player\x22\x3A\x22Kylian\x20Mbappe\x2DLottin\x22,\x22h_a\x22\x3A\x22h\x22,\x22player_id\x22\x3A\x223423\x22,\x22situation\x22\x3A\x22OpenPlay\x22,\x22season\x22\x3A\x222024\x22,\x22shotType\x22\x3A\x22RightFoot\x22,\x22match_id\x22\x3A\x2227021\x22,\x22h_team\x22\x3A\x22Real\x20Madrid\x22,\x22a_team\x22\x3A\x22Real\x20Betis\x22,\x22h_goals\x22\x3A\x222\x22,\x22a_goals\x22\x3A\x220\x22,\x22date\x22\x3A\x222024\x2D09\x2D01\x2019\x3A30\x3A00\x22,\x22player_assisted\x22\x3A\x22Ferland\x20Mendy\x22,\x22lastAction\x22\x3A\x22Pass\x22\x7D,\x7B\x22id\x22\x3A\x22590241\x22,\x22minute\x

In [32]:
strings = scripts[1].string
strings

"\n\tvar shotsData \t= JSON.parse('\\x7B\\x22h\\x22\\x3A\\x5B\\x7B\\x22id\\x22\\x3A\\x22590240\\x22,\\x22minute\\x22\\x3A\\x2212\\x22,\\x22result\\x22\\x3A\\x22BlockedShot\\x22,\\x22X\\x22\\x3A\\x220.79\\x22,\\x22Y\\x22\\x3A\\x220.7280000305175781\\x22,\\x22xG\\x22\\x3A\\x220.02347392961382866\\x22,\\x22player\\x22\\x3A\\x22Kylian\\x20Mbappe\\x2DLottin\\x22,\\x22h_a\\x22\\x3A\\x22h\\x22,\\x22player_id\\x22\\x3A\\x223423\\x22,\\x22situation\\x22\\x3A\\x22OpenPlay\\x22,\\x22season\\x22\\x3A\\x222024\\x22,\\x22shotType\\x22\\x3A\\x22RightFoot\\x22,\\x22match_id\\x22\\x3A\\x2227021\\x22,\\x22h_team\\x22\\x3A\\x22Real\\x20Madrid\\x22,\\x22a_team\\x22\\x3A\\x22Real\\x20Betis\\x22,\\x22h_goals\\x22\\x3A\\x222\\x22,\\x22a_goals\\x22\\x3A\\x220\\x22,\\x22date\\x22\\x3A\\x222024\\x2D09\\x2D01\\x2019\\x3A30\\x3A00\\x22,\\x22player_assisted\\x22\\x3A\\x22Ferland\\x20Mendy\\x22,\\x22lastAction\\x22\\x3A\\x22Pass\\x22\\x7D,\\x7B\\x22id\\x22\\x3A\\x22590241\\x22,\\x22minute\\x22\\x3A\\x2219\\x22,\\x2

In [33]:
# strip symbols to get only the json data
ind_start = strings.index("('")+2
ind_end = strings.index("')")

json_data = strings[ind_start:ind_end]
json_data = json_data.encode('utf8').decode('unicode_escape')

data = json.loads(json_data)

In [34]:
data

{'h': [{'id': '590240',
   'minute': '12',
   'result': 'BlockedShot',
   'X': '0.79',
   'Y': '0.7280000305175781',
   'xG': '0.02347392961382866',
   'player': 'Kylian Mbappe-Lottin',
   'h_a': 'h',
   'player_id': '3423',
   'situation': 'OpenPlay',
   'season': '2024',
   'shotType': 'RightFoot',
   'match_id': '27021',
   'h_team': 'Real Madrid',
   'a_team': 'Real Betis',
   'h_goals': '2',
   'a_goals': '0',
   'date': '2024-09-01 19:30:00',
   'player_assisted': 'Ferland Mendy',
   'lastAction': 'Pass'},
  {'id': '590241',
   'minute': '19',
   'result': 'SavedShot',
   'X': '0.9119999694824219',
   'Y': '0.465',
   'xG': '0.08884600549936295',
   'player': 'Eder Militão',
   'h_a': 'h',
   'player_id': '7829',
   'situation': 'FromCorner',
   'season': '2024',
   'shotType': 'Head',
   'match_id': '27021',
   'h_team': 'Real Madrid',
   'a_team': 'Real Betis',
   'h_goals': '2',
   'a_goals': '0',
   'date': '2024-09-01 19:30:00',
   'player_assisted': 'Rodrygo',
   'lastActio

In [40]:
len(data)

2

In [51]:
x = []
y = []
xG = []
result = []
team = []
data_away = data['a']
data_home = data['h']

for index in range(len(data_home)):
    for key in data_home[index]:
        if key == 'X':
            x.append(data_home[index][key])
        if key == 'Y':
            y.append(data_home[index][key])
        if key == 'h_team':
            team.append(data_home[index][key])
        if key == 'xG':
            xG.append(data_home[index][key])
        if key == 'result':
            result.append(data_home[index][key])

for index in range(len(data_away)):
    for key in data_away[index]:
        if key == 'X':
            x.append(data_away[index][key])
        if key == 'Y':
            y.append(data_away[index][key])
        if key == 'a_team':
            team.append(data_away[index][key])
        if key == 'xG':
            xG.append(data_away[index][key])
        if key == 'result':
            result.append(data_away[index][key])

In [54]:
col_names = ['x','y','xG','result','team']
df = pd.DataFrame([x,y,xG,result,team],index=col_names)


In [55]:
df

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
x,0.79,0.9119999694824219,0.97,0.9180000305175782,0.7569999694824219,0.955999984741211,0.8490000152587891,0.7530000305175781,0.855,0.855,...,0.925,0.9480000305175781,0.914000015258789,0.8919999694824219,0.8030000305175782,0.745999984741211,0.9109999847412109,0.8690000152587891,0.785,0.9080000305175782
y,0.7280000305175781,0.465,0.40900001525878904,0.5729999923706055,0.36200000762939455,0.3120000076293945,0.46799999237060547,0.40900001525878904,0.6809999847412109,0.6890000152587891,...,0.5579999923706055,0.3029999923706055,0.4520000076293945,0.54,0.7119999694824218,0.33599998474121096,0.705,0.4490000152587891,0.585,0.43200000762939456
xG,0.02347392961382866,0.08884600549936295,0.12322286516427994,0.09612398594617844,0.02109432965517044,0.04467800259590149,0.09498214721679688,0.016862040385603905,0.03648541867733002,0.035832006484270096,...,0.36057227849960327,0.044756028801202774,0.08419843763113022,0.03775697946548462,0.024000857025384903,0.020482445135712624,0.06445431709289551,0.019246773794293404,0.051805127412080765,0.08587626367807388
result,BlockedShot,SavedShot,MissedShots,SavedShot,SavedShot,MissedShots,BlockedShot,MissedShots,BlockedShot,BlockedShot,...,MissedShots,BlockedShot,BlockedShot,MissedShots,BlockedShot,SavedShot,MissedShots,SavedShot,BlockedShot,BlockedShot
team,Real Madrid,Real Madrid,Real Madrid,Real Madrid,Real Madrid,Real Madrid,Real Madrid,Real Madrid,Real Madrid,Real Madrid,...,Real Betis,Real Betis,Real Betis,Real Betis,Real Betis,Real Betis,Real Betis,Real Betis,Real Betis,Real Betis


In [56]:
df = df.T
df

,x,y,xG,result,team
0,0.79,0.7280000305175781,0.02347392961382866,BlockedShot,Real Madrid
1,0.9119999694824219,0.465,0.08884600549936295,SavedShot,Real Madrid
2,0.97,0.40900001525878904,0.12322286516427994,MissedShots,Real Madrid
3,0.9180000305175782,0.5729999923706055,0.09612398594617844,SavedShot,Real Madrid
4,0.7569999694824219,0.36200000762939455,0.02109432965517044,SavedShot,Real Madrid
5,0.955999984741211,0.3120000076293945,0.04467800259590149,MissedShots,Real Madrid
6,0.8490000152587891,0.46799999237060547,0.09498214721679688,BlockedShot,Real Madrid
7,0.7530000305175781,0.40900001525878904,0.016862040385603905,MissedShots,Real Madrid
8,0.855,0.6809999847412109,0.03648541867733002,BlockedShot,Real Madrid
9,0.855,0.6890000152587891,0.035832006484270096,BlockedShot,Real Madrid


### Option 2

In [9]:
# Importing the necessary libraries
import re
import json
import requests

from bs4 import BeautifulSoup

import pandas as pd

In [11]:
# We'll start by scraping the data of a single match
url = 'https://understat.com/match/27021'

under_response = requests.get(url)
print(under_response.status_code)

200


In [12]:
# We'll use BeautifulSoup to parse the HTML content of the page
# I also any time I have to do something besides using css selectors,
# I'll use regex to scrape the data thus I create an "ugly_soup" variable
# which is the string representation of the soup object

under_soup = BeautifulSoup(under_response.content, 'html.parser')
ugly_soup = str(soup)

In [13]:
ugly_soup

'<!DOCTYPE html>\n\n<html>\n<head>\n<base href="https://understat.com/"/>\n<title>Real Madrid 2 - 0 Real Betis (September 01 2024) | La liga | 2024/2025 | xG | Understat.com</title>\n<meta charset="utf-8"/>\n<meta content="Real Madrid 2 - 0 Real Betis. Check out detailed player statistic, goals, assists, key passes, xG, shot map, xGplot." name="description"/>\n<meta content="Real Madrid, Real Betis, La liga, 2024/2025, (September 01 2024), xG, expected goals, shot map" name="Keywords"/>\n<link href="apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>\n<link href="favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>\n<link href="favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>\n<link href="manifest.json" rel="manifest"/>\n<link color="#5bbad5" href="safari-pinned-tab.svg" rel="mask-icon"/>\n<meta content="understat" name="apple-mobile-web-app-title"/>\n<meta content="understat" name="application-name"/>\n<meta content="#ffffff" name="theme-color"/>\n<m

In [14]:
# strip unnecessary symbols and get only JSON data
shots_data = re.search("var shotsData .*= JSON.parse\('(.*)'\)", ugly_soup).group(1)
data = json.loads(shots_data.encode('utf8').decode('unicode_escape'))

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
/tmp/ipykernel_12417/763542153.py:2: SyntaxWarning: invalid escape sequence '\('
  shots_data = re.search("var shotsData .*= JSON.parse\('(.*)'\)", ugly_soup).group(1)


In [15]:
data

{'h': [{'id': '590240',
   'minute': '12',
   'result': 'BlockedShot',
   'X': '0.79',
   'Y': '0.7280000305175781',
   'xG': '0.02347392961382866',
   'player': 'Kylian Mbappe-Lottin',
   'h_a': 'h',
   'player_id': '3423',
   'situation': 'OpenPlay',
   'season': '2024',
   'shotType': 'RightFoot',
   'match_id': '27021',
   'h_team': 'Real Madrid',
   'a_team': 'Real Betis',
   'h_goals': '2',
   'a_goals': '0',
   'date': '2024-09-01 19:30:00',
   'player_assisted': 'Ferland Mendy',
   'lastAction': 'Pass'},
  {'id': '590241',
   'minute': '19',
   'result': 'SavedShot',
   'X': '0.9119999694824219',
   'Y': '0.465',
   'xG': '0.08884600549936295',
   'player': 'Eder Militão',
   'h_a': 'h',
   'player_id': '7829',
   'situation': 'FromCorner',
   'season': '2024',
   'shotType': 'Head',
   'match_id': '27021',
   'h_team': 'Real Madrid',
   'a_team': 'Real Betis',
   'h_goals': '2',
   'a_goals': '0',
   'date': '2024-09-01 19:30:00',
   'player_assisted': 'Rodrygo',
   'lastActio

In [57]:
# Turn it into a dataframe
home_df = pd.DataFrame(data['h'])
away_df = pd.DataFrame(data['a'])

In [61]:
away_df

,id,minute,result,X,Y,xG,player,h_a,player_id,situation,season,shotType,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction
0,590236,8,BlockedShot,0.8680000305175781,0.7219999694824218,0.038279157131910324,Romain Perraud,a,6500,OpenPlay,2024,LeftFoot,27021,Real Madrid,Real Betis,2,0,2024-09-01 19:30:00,Abdessamad Ezzalzouli,Pass
1,590237,9,MissedShots,0.925,0.5579999923706055,0.36057227849960327,Abdessamad Ezzalzouli,a,10095,OpenPlay,2024,Head,27021,Real Madrid,Real Betis,2,0,2024-09-01 19:30:00,Romain Perraud,Cross
2,590238,10,BlockedShot,0.9480000305175781,0.3029999923706055,0.044756028801202774,Aitor Ruibal,a,6427,OpenPlay,2024,RightFoot,27021,Real Madrid,Real Betis,2,0,2024-09-01 19:30:00,Rodrigo Sánchez,Pass
3,590239,11,BlockedShot,0.914000015258789,0.4520000076293945,0.08419843763113022,Marc Roca,a,5086,FromCorner,2024,Head,27021,Real Madrid,Real Betis,2,0,2024-09-01 19:30:00,Rodrigo Sánchez,Cross
4,590251,41,MissedShots,0.8919999694824219,0.54,0.03775697946548462,Abdessamad Ezzalzouli,a,10095,OpenPlay,2024,OtherBodyPart,27021,Real Madrid,Real Betis,2,0,2024-09-01 19:30:00,Marc Roca,Aerial
5,590252,46,BlockedShot,0.8030000305175782,0.7119999694824218,0.024000857025384903,Romain Perraud,a,6500,OpenPlay,2024,LeftFoot,27021,Real Madrid,Real Betis,2,0,2024-09-01 19:30:00,Pablo Fornals,Pass
6,590258,53,SavedShot,0.745999984741211,0.33599998474121096,0.020482445135712624,Rodrigo Sánchez,a,8410,OpenPlay,2024,LeftFoot,27021,Real Madrid,Real Betis,2,0,2024-09-01 19:30:00,Pablo Fornals,Pass
7,590260,57,MissedShots,0.9109999847412109,0.705,0.06445431709289551,Abdessamad Ezzalzouli,a,10095,OpenPlay,2024,LeftFoot,27021,Real Madrid,Real Betis,2,0,2024-09-01 19:30:00,Johnny Cardoso,Pass
8,590261,60,SavedShot,0.8690000152587891,0.4490000152587891,0.019246773794293404,Vitor Roque,a,12302,OpenPlay,2024,Head,27021,Real Madrid,Real Betis,2,0,2024-09-01 19:30:00,Youssouf Sabaly,Aerial
9,590263,62,BlockedShot,0.785,0.585,0.051805127412080765,Pablo Fornals,a,2335,OpenPlay,2024,RightFoot,27021,Real Madrid,Real Betis,2,0,2024-09-01 19:30:00,Vitor Roque,Pass


In [58]:
print(home_df.iloc[0])

id                               590240
minute                               12
result                      BlockedShot
X                                  0.79
Y                    0.7280000305175781
xG                  0.02347392961382866
player             Kylian Mbappe-Lottin
h_a                                   h
player_id                          3423
situation                      OpenPlay
season                             2024
shotType                      RightFoot
match_id                          27021
h_team                      Real Madrid
a_team                       Real Betis
h_goals                               2
a_goals                               0
date                2024-09-01 19:30:00
player_assisted           Ferland Mendy
lastAction                         Pass
Name: 0, dtype: object
